In [1]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
zhipuai.api_key = os.environ['ZHIPUAI_API_KEY']

# 加载 PDF
loaders_chinese = [
    PyMuPDFLoader("/workspaces/LLM_Project/database/knowledge_db/pumpkin_book.pdf") # 南瓜书
    # 大家可以自行加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)

embedding = ZhipuAIEmbeddings()

persist_directory = '../../data_base/vector_db/chroma'

!rm -rf '../../data_base/vector_db/chroma'

In [2]:
vectordb = Chroma.from_documents(
    documents=split_docs[:100], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [3]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：100


In [4]:
question="什么是机器学习"

sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [5]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
检索到的第1个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
12.7.4 式 (12.60) 的推导 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
149
12.7.5 经验损失最小化 . . . . . . . . . . . . . . . . . . . . . . 
--------------
检索到的第2个内容: 
45
5.5.3
式 (5.22) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
45
5.5.4
式 (5.23) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
45
5.6
深
--------------


In [6]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [7]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
MMR 检索到的第1个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
12.7.4 式 (12.60) 的推导 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
149
12.7.5 经验损失最小化 . . . . . . . . . . . . . . . . . . . . . . 
--------------
MMR 检索到的第2个内容: 
46
5.6.2
深度学习的起源 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
46
5.6.3
怎么理解特征学习
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
46
第 6 章 支持向量机
47
--------------


In [8]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModel

llm = ZhipuAILLM(model="chatglm_std", temperature=0)

# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = " 2025 年大语言模型效果最好的是哪个模型"

/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
result = qa_chain({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

LLM 对问题的回答：" 无法预测2025年大语言模型效果最好的是哪个模型，因为这是未来的事情，而且受到许多因素的影响，如技术进步、数据规模和质量等。但是，可以期待未来会有更先进、更高效的大语言模型出现。感谢您的提问！"


In [10]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

向量数据库检索到的最相关的文档：page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第 1 章和第 2 章的公式强烈不建议深究，简单过一下即可，等你学得' metadata={'source': '/workspaces/LLM_Project/database/knowledge_db/pumpkin_book.pdf', 'file_path': '/workspaces/LLM_Project/database/knowledge_db/pumpkin_book.pdf', 'page': 1, 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20231117152045-00'00'", 'modDate': '', 'trapped': ''}
